In [1]:
%matplotlib tk
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from io import StringIO
import requests
import matplotlib as mpl

In [2]:
WISC_2020_TWO_PARTY_VOTE = 0.006381
WISC_2022_TWO_PARTY_VOTE = 0.034347

SIGNATURE = 'By Will Bradley, 2023'

In [3]:
# Galen Metzger suggested using 2022 ballots returned to normalize instead of voter registration
ballots_2022_df = pd.read_csv('AbsenteeCounts_Muni__25.csv', index_col='HINDI', usecols=['BallotsReturned', 'HINDI'])\
    .rename(columns={'BallotsReturned': 'BallotsReturned2022'})\
    .drop(99999) # TOTAL column

gdf = gpd.read_file('WI_Cities%2C_Towns_and_Villages_(January_2023)') # https://data-ltsb.opendata.arcgis.com/datasets/2d13492a59a24dd0ba990abf1f86800f_0/explore?location=43.322926%2C-89.269110%2C8.58

In [4]:
results_20 = pd.read_csv('WISC_PRES_2020_by_jurisdiction.csv')
results_22 = pd.read_csv('WISC_GOV_2022_by_jurisdiction.csv')

In [5]:
EARLY_VOTE_URL = 'https://elections.wi.gov/sites/default/files/documents/AbsenteeCounts_Muni_2023%20Spring%20Election_11.csv'

In [6]:
results = pd.merge(results_20, results_22, on=['county', 'ctv', 'jurisdiction'], suffixes=['_20', '_22'])

In [7]:
ballots_df = pd.read_csv(StringIO(requests.get(EARLY_VOTE_URL).text))
voting_merged_df = pd.merge(ballots_df, ballots_2022_df, on='HINDI')
voting_merged_df['HINDI'] = voting_merged_df['HINDI'].apply(lambda n: str(n).zfill(5))

merged = gdf.merge(voting_merged_df, left_on='DOA', right_on='HINDI', how='left')
merged.set_index('GEOID', drop=True, inplace=True)

merged['returned_rate'] = merged['BallotsReturned'] / merged['BallotsReturned2022']

In [8]:
merged['CNTY_NAME'] = merged['CNTY_NAME'].str.upper()
merged['MCD_NAME'] = merged['MCD_NAME'].str.title()
merged['CTV'] = merged['CTV'].str.upper()

In [9]:
merged_with_results = pd.merge(merged, results, left_on=['CNTY_NAME', 'MCD_NAME', 'CTV'], right_on=['county', 'jurisdiction', 'ctv'])
#merged_with_results = merged_with_results[merged_with_results['BallotsReturned'] >= 100]
#merged_with_results['ballots_in'] = merged_with_results['BallotsReturned'] / merged_with_results['BallotsSent']

def _compute_margin(d: pd.Series, r: pd.Series) -> pd.Series:
    return (d - r) / (d + r)

merged_with_results['D_margin_20'] = _compute_margin(merged_with_results['DEM_20'], merged_with_results['REP_20'])
merged_with_results['D_margin_22'] = _compute_margin(merged_with_results['DEM_22'], merged_with_results['REP_22'])
merged_with_results['D_vote_margin_20'] = merged_with_results['DEM_20'] - merged_with_results['REP_20']
merged_with_results['D_vote_margin_22'] = merged_with_results['DEM_22'] - merged_with_results['REP_22']

In [10]:
statewide_return_rate = merged_with_results['BallotsReturned'].sum() / merged_with_results['BallotsReturned2022'].sum()
statewide_return_rate

0.47594057270592766

In [11]:
CMAP = mpl.colors.LinearSegmentedColormap.from_list("political", ["red","purple","blue"])

In [12]:
plt.close('all')

In [13]:
fig1, ax1 = plt.subplots()
ax1.scatter(
    merged_with_results['D_margin_22'] * 100,
    merged_with_results['returned_rate'], 
    s=merged_with_results['BallotsReturned'] / 10,
    alpha=0.35,
    c=merged_with_results['D_margin_22'],
    cmap=CMAP,
    norm=mpl.colors.Normalize(-0.7, 0.7)
)
ax1.axhline(statewide_return_rate, color='red', lw=0.5)
ax1.axvline(WISC_2022_TWO_PARTY_VOTE, color='black', lw=0.5)
ax1.set_ylim(0, 1.2)
ax1.set_xlabel('2022 governor Democratic margin, %')
ax1.set_ylabel('Ballots returned / ballots returned the 2022 fall election')
ax1.set_title('Return rate over 2022 Wisconsin governor result by municipality\n(area proportonal to 2023 ballots returned)')
ax1.text(0.01, statewide_return_rate / 1.2, 'statewide return rate', ha='left', va='bottom', transform=ax1.transAxes)

Text(0.01, 0.3966171439216064, 'statewide return rate')

In [14]:
fig2, ax2 = plt.subplots()
trend = (merged_with_results['D_margin_22'] - merged_with_results['D_margin_20']) \
    - (WISC_2022_TWO_PARTY_VOTE - WISC_2020_TWO_PARTY_VOTE)
ax2.scatter(
    trend * 100,
    merged_with_results['returned_rate'], 
    s=merged_with_results['BallotsReturned'] / 10,
    alpha=0.35,
    c=trend,
    cmap=CMAP,
    norm=mpl.colors.Normalize(-0.1, 0.1)
)
ax2.axvline(0, color='black', lw=0.5)
ax2.axhline(statewide_return_rate, color='red', lw=0.5)
ax2.set_xlim(-10, 10)
ax2.set_ylim(0, 1.2)
ax2.set_xlabel('2020 president - 2022 governor trend, %')
ax2.set_ylabel('Ballots returned / ballots returned the 2022 fall election')
ax2.set_title('Return rate over 2020 - 2022 trend by municipality\n(area proportonal to 2023 ballots returned)')
ax2.text(0.01, statewide_return_rate / 1.2, 'statewide return rate', ha='left', va='bottom', transform=ax2.transAxes)

Text(0.01, 0.3966171439216064, 'statewide return rate')

In [15]:
fig3, ax3 = plt.subplots()
ax3.scatter(
    merged_with_results['D_vote_margin_22'],
    merged_with_results['returned_rate'], 
    s=merged_with_results['BallotsReturned'] / 10,
    alpha=0.35,
    c=merged_with_results['D_vote_margin_22'],
    cmap=CMAP,
    norm=mpl.colors.SymLogNorm(100, vmin=-1000000, vmax=1000000)
)
ax3.axvline(0, color='black', lw=0.5)
ax3.axhline(statewide_return_rate, color='red', lw=0.5)
#ax3.set_xlim(-10000, 1000000)
ax3.set_xscale('symlog', linthresh=1000)
ax3.set_ylim(0, 1.2)
ax3.set_xlabel('2022 raw vote margin')
ax3.set_ylabel('Ballots returned / ballots returned the 2022 fall election')
ax3.set_title('Return rate over 2022 governor Democratic raw vote margin by municipality\n(area proportonal to 2023 ballots returned)')
ax3.text(0.01, statewide_return_rate / 1.2, 'statewide return rate', ha='left', va='bottom', transform=ax3.transAxes)

Text(0.01, 0.3966171439216064, 'statewide return rate')

In [16]:
(merged_with_results['D_vote_margin_22'] - merged_with_results['D_vote_margin_20']).describe()

count     1901.000000
mean        37.395055
std        825.659071
min     -35127.000000
25%         13.000000
50%         35.000000
75%         71.000000
max       1357.000000
dtype: float64

In [17]:
fig4, ax4 = plt.subplots()
ax4.scatter(
    merged_with_results['D_vote_margin_22'] - merged_with_results['D_vote_margin_20'],
    merged_with_results['returned_rate'], 
    s=merged_with_results['BallotsReturned'] / 10,
    alpha=0.35,
    c=merged_with_results['D_margin_22'],
    cmap=CMAP,
    norm=mpl.colors.Normalize(-1, 1)
)
ax4.axvline(0, color='black', lw=0.5)
ax4.axhline(statewide_return_rate, color='red', lw=0.5)
#ax4.set_xlim(-100000, 10000)
ax4.set_xscale('symlog', linthresh=100)
ax4.set_ylim(0, 1.2)
ax4.set_xlabel('Change in raw vote margin, 2020 president - 2022 governor')
ax4.set_ylabel('Ballots returned / ballots returned the 2022 fall election')
ax4.set_title('Return rate over change in Democratic raw vote margin, 2020 - 2022 by municipality\n(area proportonal to 2023 ballots returned, colored by 2022 governor result)')
ax4.text(0.01, statewide_return_rate / 1.2, 'statewide return rate', ha='left', va='bottom', transform=ax4.transAxes)

Text(0.01, 0.3966171439216064, 'statewide return rate')

In [18]:
for i, fig in enumerate((fig1, fig2, fig3, fig4)):
    fig.set_size_inches(12, 8)
    fig.text(0.01, 0.01, SIGNATURE)

    fig.savefig(f'output/scatter_{i}.png', dpi='figure')

In [19]:
merged_with_results[merged_with_results['jurisdiction'] == 'Cedarburg']

,OBJECTID,CNTY_FIPS,CNTY_NAME,COUSUBFP,MCD_FIPS,MCD_NAME,CTV,WARD_FIPS,WARDID,SUPERID,...,Total Votes Cast_20,DEM_20,REP_20,Total Votes Cast_22,DEM_22,REP_22,D_margin_20,D_margin_22,D_vote_margin_20,D_vote_margin_22
1177,1185,55089,OZAUKEE,13375,5508913375,Cedarburg,C,None,None,None,...,8193,4032,4013,7213,3564,3585,0.002362,-0.002937,19,-21
1178,1186,55089,OZAUKEE,13400,5508913400,Cedarburg,T,None,None,None,...,4544,1653,2832,3938,1425,2483,-0.262876,-0.270727,-1179,-1058
